In [1]:
import pandas, re, cleantext, nltk
import pandas as pd
from itertools import chain
from nltk.corpus import stopwords

/tmp/ipykernel_34865/2493697506.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas, re, cleantext, nltk
Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [2]:
def clean(csv):
  df = pandas.read_csv(csv)
  for i in range(len(df["content"])):
    df.loc[i, "content"] = cleantext.clean(df.loc[i, "content"],
      lower=True,
      no_line_breaks=True,
      no_numbers=True,
      no_emails=True,
      no_urls=True,
      replace_with_number="<NUM>")
  return df

clean("news_sample.csv").to_csv("cleaned_news_sample.csv")

In [3]:
from nltk import regexp_tokenize

def tokenize(stringInput):
              
    return regexp_tokenize(stringInput, pattern=r"\s|[.,;']", gaps=True)


In [4]:
def stem(input):
  stemmer = nltk.SnowballStemmer(language="english", ignore_stopwords=True)
  stem_words = []
  for token in input:
    x = stemmer.stem(token)
    stem_words.append(x)
  return stem_words

In [5]:
def remove_stopwords(cell):
  for token in cell:
    stop_words = stopwords.words('english')
    filteredInput = []
    
    for token in cell:
      if token not in stop_words:
        filteredInput.append(token)
        
    return filteredInput

In [6]:
def run():
  data = "news_sample.csv"
  cleaned = clean(data)
  tokenizedData = tokenize(cleaned)
  stopwordsRemoved = remove_stopwords(tokenizedData)
  StemmedData = stem(stopwordsRemoved)
  
  for x in StemmedData:
    print(x)

run()

ValueError: Must have equal len keys and value when setting with an iterable